In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

# Load your CSVimport pandas as pd

columns = [
    "Acc_Date",
    "Acc_No",
    "Title",
    "ISBN",
    "Author_Editor",
    "Edition_Volume",
    "Place_Publisher",
    "Year",
    "Pages",
    "Class_No"
]

df = pd.read_csv(
    "dau_library_data.csv",
    usecols=range(len(columns)),  
    names=columns,
    header=0,
    encoding='latin1'
)

df.shape

In [ ]:
df=df.drop_duplicates(subset=["Title",
    "ISBN",
    "Author_Editor",
    "Edition_Volume",
    "Place_Publisher",
    "Year",
    "Pages",
    "Class_No"])
df.shape

In [ ]:
# open Library Data fetch through ISBN 5000 rows 
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

df10 = df.iloc[:5000].copy()

descriptions = []

headers = {
    "User-Agent": "Mozilla/5.0 (compatible; Sahil-OpenLibrary-PTag/Debug)"
}

for i, row in df10.iterrows():
    isbn = str(row.get("ISBN", "")).strip()
    print(f"\nRow {i+1} | ISBN: {isbn}")

    desc = "ISBN Not Matched"

    if isbn:
        url = f"https://openlibrary.org/isbn/{isbn}"
        try:
            r = requests.get(url, headers=headers, timeout=10, allow_redirects=True)

            if r.status_code == 200:
                print("ISBN page found ✅")

                soup = BeautifulSoup(r.text, "html.parser")
                p_tag = soup.select_one(
                    "div.book-description div.read-more__content p"
                )

                if p_tag:
                    desc = p_tag.get_text(strip=True)
                    print("Description extracted ✅")
                else:
                    desc = "Description Not Available"
                    print("Description missing ❌")

            else:
                
                print("ISBN not found ❌")

        except Exception as e:
            print("Error:", e)

    else:
        print("ISBN missing in CSV ❌")

    descriptions.append(desc)
    time.sleep(1)

df10["description"] = descriptions
df10.to_csv("OpenLibrary_5000_rows.csv", index=False)

print("\nSaved CSV with correct description labels ✅")


In [ ]:
#HTML tag Through Google books throguh ISBN 

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
 
df100 = df

descriptions = []

headers = {
    "User-Agent": "Mozilla/5.0"
}

def clean_isbn(isbn):
    return re.sub(r"[^0-9Xx]", "", str(isbn))

for i, isbn in enumerate(df100["ISBN"]):
    isbn = clean_isbn(isbn)
    print(f"Processing Row {i+1} | ISBN: {isbn}")

    desc = "Not Found"

    if isbn:
        url = f"https://books.google.com/books?vid=ISBN{isbn}"

        try:
            r = requests.get(url, headers=headers, timeout=10)
            soup = BeautifulSoup(r.text, "html.parser")

            desc_div = soup.find("div", id="synopsis")
            if desc_div:
                desc = desc_div.get_text(separator=" ", strip=True)

        except Exception as e:
            print("Error:", e)

    descriptions.append(desc)
    time.sleep(0.2)

df100["description"] = descriptions
df100.to_csv("HTML_tag_through_All_36000.csv", index=False)

print("\nSaved first 100 rows with descriptions ✅")


In [ ]:
import pandas as pd
df=pd.read_csv('OpenLibrary_5000_rows.csv') 
isbn_not_matched = (df["description"] == "ISBN Not Matched").sum()
desc_not_available = (df["description"] == "Description Not Available").sum()

print("Desc Not availble out of 5000 row",desc_not_available)  
print("ISBN not found out of 5000 row",isbn_not_matched)


In [ ]:
import pandas as pd
df=pd.read_csv('HTML_tag_through_All_36000.csv') 
isbn_not_matched = (df["description"] == "Not Found").sum() 
print("not found In HTML tag through ",isbn_not_matched)

In [ ]:
import pandas as pd

df1 = pd.read_csv("HTML_tag_through_All_36000.csv")
df2 = pd.read_csv("OpenLibrary_5000_rows.csv")

df = pd.DataFrame({
    "desc1": df1["description"],
    "desc2": df2["description"]
})

missing_vals = ["Not Found", "ISBN Not Matched", "Description Not Available"]
 
both_missing = (df["desc1"].isin(missing_vals) & df["desc2"].isin(missing_vals)).sum()
 
left_missing_right_has = (df["desc1"].isin(missing_vals) & ~df["desc2"].isin(missing_vals)).sum()
 
right_missing_left_has = (df["desc2"].isin(missing_vals) & ~df["desc1"].isin(missing_vals)).sum()
 
both_same = (df["desc1"] == df["desc2"]).sum()
 
both_same_missing = (
    (df["desc1"] == df["desc2"]) & df["desc1"].isin(missing_vals)
).sum()

print("Both Missing:", both_missing)
print("Left Missing but Right Has:", left_missing_right_has)
print("Right Missing but Left Has:", right_missing_left_has)
print("Both Descriptions Same:", both_same)
print("Both Same & Missing:", both_same_missing)


In [ ]:
import pandas as pd

html_df = pd.read_csv("HTML_tag_through_All_36000.csv")
ol_df = pd.read_csv("OpenLibrary_5000_rows.csv")

missing_vals = ["Not Found", "ISBN Not Matched", "Description Not Available"]
 
ol_desc_aligned = ol_df["description"].reindex(html_df.index)
 
mask = html_df["description"].isin(missing_vals) & ~ol_desc_aligned.isin(missing_vals)

html_df.loc[mask, "description"] = ol_desc_aligned[mask]

html_df.to_csv("Final_Merged_Descriptions.csv", index=False)

print("Merged successfully Sahil ✅")


In [ ]:
import pandas as pd

df1 = pd.read_csv("Final_Merged_Descriptions.csv")
df2 = pd.read_csv("OpenLibrary_5000_rows.csv")

df = pd.DataFrame({
    "desc1": df1["description"],
    "desc2": df2["description"]
})

missing_vals = ["Not Found", "ISBN Not Matched", "Description Not Available"]
 
both_missing = (df["desc1"].isin(missing_vals) & df["desc2"].isin(missing_vals)).sum()
 
left_missing_right_has = (df["desc1"].isin(missing_vals) & ~df["desc2"].isin(missing_vals)).sum()
 
right_missing_left_has = (df["desc2"].isin(missing_vals) & ~df["desc1"].isin(missing_vals)).sum()
 
both_same = (df["desc1"] == df["desc2"]).sum()
 
both_same_missing = (
    (df["desc1"] == df["desc2"]) & df["desc1"].isin(missing_vals)
).sum()

print("Both Missing:", both_missing)
print("Left Missing but Right Has:", left_missing_right_has)
print("Right Missing but Left Has:", right_missing_left_has)
print("Both Descriptions Same:", both_same)
print("Both Same & Missing:", both_same_missing)


In [ ]:
# Title + Author through Gooogle Bookss

import pandas as pd
import requests
import re
import time
from urllib.parse import quote_plus
 
df = pd.read_csv("Final_Merged_descriptions.csv")

original_nf = df[
    (df["description"].isna()) |
    (df["description"].astype(str).str.strip() == "") |
    (df["description"] == "Not Found")
].copy()

original_nf=original_nf[:50].copy() 
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower().strip()
    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text

df["clean_title"] = df["Title"].apply(clean_text)
df["clean_author"] = df["Author_Editor"].apply(clean_text)

 
if "Publisher" in df.columns:
    df["clean_publisher"] = df["Publisher"].apply(clean_text)
else:
    df["clean_publisher"] = ""

 
def google_books_api_search(query):
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}&maxResults=1"
    try:
        res = requests.get(url, timeout=10).json()
        items = res.get("items")
        if not items:
            return None
        return items[0].get("volumeInfo", {}).get("description")
    except:
        return None

 
def get_google_books_description(title, author, publisher):
    queries = []

    
    if author:
        queries.append(f"intitle:{title}+inauthor:{author}")

   
    if author:
        queries.append(f"intitle:{title}+{author}")

    
    if publisher:
        queries.append(f"intitle:{title}+inpublisher:{publisher}")

     
    queries.append(f"intitle:{title}")

    for q in queries:
        desc = google_books_api_search(quote_plus(q))
        if desc and len(desc) > 50:
            return desc

    return None

 
for idx, row in original_nf.iterrows():

    print("Processing:", row["Title"])

    desc = get_google_books_description(
        df.at[idx, "clean_title"],
        df.at[idx, "clean_author"],
        df.at[idx, "clean_publisher"]
    )

    if desc:
        df.at[idx, "description"] = desc
    else:
        df.at[idx, "description"] = "Not Found"

    time.sleep(0.2)

 
final_output = original_nf[["Title", "Author_Editor"]].copy()
final_output["description"] = df.loc[original_nf.index, "description"]

final_output.to_csv("Final_GoogleBooks_Descriptions.csv", index=False)

print("All done Sahil ✅ Google Books description extracted successfully")


In [ ]:
# match with Final_GoogleBooks_Descriptions.csv and copy into Final_Merged_Descriptions.csv

import pandas as pd

 
source = pd.read_csv("Final_GoogleBooks_Descriptions.csv")   # has correct descriptions
target = pd.read_csv("Final_Merged_Descriptions.csv")   # has Not Found / null descriptions

 
def clean_text(text):
    if pd.isna(text):
        return ""
    return str(text).strip().lower()

for col in ["Title", "Author_Editor"]:
    source[col] = source[col].apply(clean_text)
    target[col] = target[col].apply(clean_text)

 
source = source.rename(columns={"description": "description_src"})

 
merged = target.merge(
    source[["Title", "Author_Editor", "description_src"]],
    on=["Title", "Author_Editor"],
    how="left"
)

 
mask = (
    merged["description"].isna() |
    (merged["description"].astype(str).str.strip() == "") |
    (merged["description"] == "Not Found")
)

merged.loc[mask, "description"] = merged.loc[mask, "description_src"]

 
merged = merged.drop(columns=["description_src"])

 
merged.to_csv("dau_with_description.csv", index=False)

print("Done Sahil ✅ Description updated only where match exists")


In [ ]:
df = pd.read_csv("dau_with_description.csv")
df=df.drop_duplicates(subset=["Title",
    "ISBN",
    "Author_Editor",
    "Edition_Volume",
    "Place_Publisher",
    "Year",
    "Pages",
    "Class_No"])
df.shape


(32377, 11)

In [5]:
original_nf = df[
    (df["description"].isna()) |
    (df["description"].astype(str).str.strip() == "") |
    (df["description"] == "Not Found")
].copy()
original_nf.shape

(3189, 11)

In [ ]:
import pandas as pd 
df = pd.read_csv("D:\COLLAGE\DAIICT\2 - SEM\BDE\Project\Big-Data-Engineering-\Data\processed\dau_with_description.csv")
df.info

FileNotFoundError: [Errno 2] No such file or directory: 'dau_with_description.csv'